In [1]:
import os
import pypsa
import geopandas as gpd
import pandas as pd
import numpy as np
from numpy import genfromtxt
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy
import networkx as nx
from geovoronoi import points_to_coords
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [16]:
# Load demand data
load = genfromtxt('database/input_data/loads_kalimantan.csv', delimiter=',')
# Load dataframe with total demand and corresponding week of the year
df = pd.read_csv(r'database/input_data/df_kalimantan.csv', index_col=[0])
# Load coordinates of equivalent electrical buses
buses = pd.read_csv(r'database/input_data/buses_kalimantan.csv', index_col=[0])
# Load data of the transmission lines
links= pd.read_csv(r'database/input_data/links_kalimantan.csv', index_col=[0])
# Load generator data
generators = pd.read_csv(r'database/input_data/generators_kalimantan.csv', index_col=[0])
# Load solar time series data
solar = pd.read_csv(r'database/input_data/solar_kalimantan.csv', index_col=[0])
# Load carbon emission data per carrier
carrier = pd.read_csv(r'database/input_data/carriers_kalimantan.csv', index_col=[0])
# Load onshore wind time series data
wind_os = pd.read_csv(r'database/input_data/windons_kalimantan.csv', index_col=[0])
# Load offshore wind time series data
wind_offs = pd.read_csv(r'database/input_data/windoffs_kalimantan.csv', index_col=[0])

In [17]:
# Set all capacities to zero        
generators.p_nom = 0

In [22]:
"""Define start and end timestep and modify capital cost accordingly"""
# start and end timestep (can be modified for testruns)
time_start = 0
time_end = 23
#time_end = len(load)
#now = 19
load_short = load[time_start:time_end]/(1e6)

In [19]:
# Capital cost of transmission lines
line_learning = 0.613e3

In [20]:
# Calculate total system carbon emissions 2018 (multiply by 4.11, which is the demand growth with the total production in 2018 reported by PLN)
carbon_emissions = ((100.5251e6)*(101/0.38*3.6) + ((16.0537e6)*(56/0.56*3.6)) + ((0.8754e6) *(74/0.48*3.6)))/1e3*4.11 

In [23]:
"""Create scenarios and run model"""
parametersweep = [0.8, 0.6]
scenarios=[]
for i in parametersweep:
    scenarios.append(carbon_emissions*i)

container = []
for i in range(len(scenarios)):
    # Create network
    container.append(pypsa.Network()) 
    
    # Import components
    container[i].import_components_from_dataframe(buses,"Bus")
    container[i].import_components_from_dataframe(generators,"Generator")
    container[i].import_components_from_dataframe(links, "Line")
    #container[i].import_components_from_dataframe(storage, "StorageUnit")
    container[i].import_components_from_dataframe(carrier, "Carrier")
    container[i].add("GlobalConstraint", "CO2Limit", carrier_attribute="co2_emissions", 
                     sense="<=", constant=scenarios[i]/2920*time_end)

    # Adapt time series (if not an entire year)
    wind_short = wind_os.iloc[time_start:time_end]
    wind_short = wind_short.reset_index(drop=True)
    wind_offs_short = wind_offs.iloc[time_start:time_end]
    wind_offs_short = wind_offs_short.reset_index(drop=True)
    solar_short = solar.iloc[time_start:time_end]
    solar_short = solar_short.reset_index(drop=True)
    timeseries = pd.merge(solar_short, wind_short, left_index=True, right_index=True, how="outer")
    timeseries = pd.merge(timeseries, wind_offs_short, left_index=True, right_index=True, how="outer")
    
    # Import timeseries
    container[i].import_series_from_dataframe(timeseries,"Generator","p_max_pu")
    container[i].generators_t.p_max_pu = timeseries
    
    # Set snapshots (timesteps)
    snapshots = range(len(load))
    snapshots1 = range(len(load_short))
    container[i].set_snapshots(snapshots1)
    
    # Set aggregation of timeseries by adding weight (uncomment for hourly timesteps)
    container[i].snapshot_weightings[:] = 3
    
    # Set limits for capacities transmission lines and N-1 factor
    container[i].lines.s_nom_extendable = True
    container[i].lines.s_nom_min = container[i].lines.s_nom
    contingency_factor = 0.7
    container[i].lines.s_max_pu = contingency_factor
    
    # Add load to network
    container[i].madd("Load", "Load " + container[i].buses.index, bus=buses.index, p_set=load_short)
    
    # Optimize the system, need to have a gurobi license installed 
    container[i].lopf(solver_name="glpk", solver_io='python')
    
    # Print cost results
    tot_sys = container[i].objective/(container[i].generators_t.p.sum().sum()*3)
    print("Objective:", container[i].objective, "$")
    print("Average System Cost:", tot_sys, "$/MWh")

Index(['PLTBg_Maju_Aneka_Sawit', 'PLTBg_Rea_Kaltim', 'PLTBg_Suka_Damai',
       'PLTBg_Sukajadi_Sawit_Mekar', 'PLTBg_Unggul_Lestari',
       'PLTBm_Gunung_Sari_HHM', 'PLTBm_Karangan_DL', 'PLTBm_Korintiga_Hutani',
       'PLTBm_Kotawaringin_Barat', 'PLTBm_PT._Rezeki',
       ...
       'PLTS_Kayan_Hulu_ESDM', 'PLTS_Lumbis_Ogong_ESDM', 'PLTS_Pulau_Limbung',
       'PLTS_Sebatik', 'PLTS_Sebatik_ESDM', 'PLTS_Sebuku_ESDM',
       'PLTS_Seimenggaris_ESDM', 'PLTS_Sidding', 'PLTS_Temajuk',
       'PLTS_Tulin_Onsoi_ESDM'],
      dtype='object', length=409)
ERROR:pypsa.io:Error, new components for Generator are not unique
Index(['0', '1', '3', '1', '4', '5', '5', '7', '7', '9', '10', '11', '11',
       '13', '15', '14', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '25', '29', '24', '30', '28', '31', '22', '33', '32', '32',
       '35', '36', '37', '35', '39', '40', '40', '42', '43', '44', '45', '46',
       '46', '45', '49', '50', '51', '52', '49', '54', '54', '56', '

RuntimeError: Fatal error: cannot find the component data in the owning component's _data dictionary.